In [44]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error


In [45]:
data=pd.read_csv('new_data_actual.csv')

In [86]:
data.head(5)

,Depth,WS,RH (%),Temp (Deg),PM0.23_0.30,PM0.30_0.40,PM0.40_0.50,PM0.50_0.65,PM0.65_0.80,PM0.80_1,PM1_1.6,PM1.6_2,PM2_3,PM3_4,PM4_5,PM5_7.5,PM7.5_10,PM10_15,PM15_20
0,412.0,2.500000,71.5,26.0,10.1,10.1,8.7,4.2,1.3,1.0,1.1,3.5,3.9,0.8,0.4,0.7,0.0,0.0,0.0
1,412.0,3.333333,71.3,26.0,10.3,10.4,9.0,4.4,1.4,1.1,1.1,2.5,2.6,0.6,2.4,5.0,3.0,5.7,0.0
2,412.0,2.222222,71.1,26.1,9.8,9.7,7.9,3.7,1.2,0.8,1.1,2.4,3.1,0.0,0.3,0.0,0.0,0.0,0.0
3,412.0,2.222222,71.1,26.1,10.1,10.1,8.4,4.1,1.4,1.0,1.3,3.4,5.5,1.3,1.7,1.5,1.9,2.9,0.0
4,412.0,0.833333,71.0,26.1,10.4,10.5,8.9,4.8,1.8,1.8,2.1,3.8,5.2,2.1,1.5,5.3,4.9,5.8,15.7


In [90]:
a=data.iloc[1,:4]

#preds

In [52]:
X, y = data.iloc[:,:4],data.iloc[:,4]



In [76]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

data_dmatrix

/home/krupal/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/krupal/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)


In [100]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [101]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

[02:10:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [59]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 4.589112


In [110]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=X, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

AttributeError: 'DataFrame' object has no attribute 'num_row'

In [4]:
data_points = np.array(data_points)

data_target = pd.DataFrame(data["PM0.23_0.30"])
data_target = np.array(data_target)
data_target = data_target.reshape(1,-1)[0]

data_target2 = pd.DataFrame(data["PM0.30_0.40"])
data_target2 = np.array(data_target2)
data_target2 = data_target2.reshape(1,-1)[0]

data_target3 = pd.DataFrame(data["PM0.40_0.50"])
data_target3 = np.array(data_target3)
data_target3 = data_target2.reshape(1,-1)[0]


In [13]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.utils import check_random_state
from sklearn.model_selection import StratifiedKFold

In [3]:
data_points = pd.DataFrame()
data_points["Depth"] = data["Depth"]
data_points["WS"] = data["WS"]
data_points["RH (%)"] = data["RH (%)"]
data_points["Temp (Deg)"] = data["Temp (Deg)"]

In [37]:

i=0
rng = check_random_state(i)
x_train,x_test, y_train, y_test = train_test_split(data_points,data_target,random_state=rng)
#x_train = xgb.DMatrix(x_train)
#x_test= xgb.DMatrix(x_test)
#y_train = xgb.DMatrix(y_train)
#y_test= xgb.DMatrix(y_test)

In [40]:

XGBModel = xgb.XGBRegressor()
model=XGBModel.fit(x_train,y_train , verbose=False )

[01:39:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [28]:
XGBpredictions = XGBModel.predict(x_test)


In [29]:
MAE = mean_absolute_error(y_test, XGBpredictions)
sc = r2_score(y_test, XGBpredictions)
print('Random forest validation MAE = ', MAE)
print('Random forest validation scorers= ', sc)

MsE = mean_squared_error(y_test, XGBpredictions)

print('Random forest validation MSE = ', MsE)

Random forest validation MAE =  1.2343711584934405
Random forest validation scorers=  0.22673004761247273
Random forest validation MSE =  54.706984112088584


In [31]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_accuracy = evaluate(XGBModel, data_points, data_target)

Model Performance
Average Error: 0.9881 degrees.
Accuracy = 79.27%.
